# Lab 4: Transfer Learning and BERT!

Due on: 04/15/2024 @ 9pm

Agenda
------
+ Get an overview of the BERT architecture
+ Download and play with a pre-trained BERT model
+ Train your own network that has a pre-trained BERT component
+ Play with some parameters!


Summary
----
This lab will guide you through the setup and uses of pretrained models in `Tensorflow` and `transformers`. In this lab we focus on BERT (https://github.com/google-research/bert/) an __encoder only transformer model__. We will use the pretrained model to extract features from text and use these features to train a classifier to classify different types of data. We will setup a binary classifier on the IMDB dataset. You will be working on training new models on the other datasets.

# Task 0: Who is in your group?

Please work in groups of up to 3 people!

Jack Arseneau

TASK 1: BERT questions
---

Before we dive into the code, we'd like you to take a moment to familiarize yourself with the architecture of BERT.

Take a look at the following resources as a group:
- [Illustrated BERT](http://jalammar.github.io/illustrated-bert/)
- SLP Chapter 11.1
- [BERT paper](https://aclanthology.org/N19-1423/)

Answer the questions in the next cell about the BERT architecture.

1. What does BERT stand for? __YOUR ANSWER HERE__
2. What task is the BERT pre-trained model trained on? __YOUR ANSWER HERE__
3. How many parameters does BERT_{BASE} have? BERT_{LARGE}?   __YOUR ANSWER HERE__
4. What is the architecture of BERT_{BASE}? (what kind of neural network is it, what are the relevant dimensionality numbers?)  __YOUR ANSWER HERE__
5. What can we feed as input to BERT?  __YOUR ANSWER HERE__
6. What is the output of BERT?  __YOUR ANSWER HERE__
7. If we want to use BERT to help with a classification task, what is a high-level description of what we'll do? (2 - 3 sentences.)  __YOUR ANSWER HERE__

Installation Intermission
----

In [1]:
# make sure that the correct version of transformers and tensorflow is installed
# https://huggingface.co/docs/transformers/index
# https://www.tensorflow.org/install/pip
!pip install tensorflow==2.14.0 transformers==4.35.0

# if you are on a mac w/ an M1 chip, you'll need a specific tensorflow-metal version
# !pip install tensorflow-metal==1.1.0

ERROR: Could not find a version that satisfies the requirement tensorflow==2.14.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0)
ERROR: No matching distribution found for tensorflow==2.14.0


In [ ]:
import tensorflow as tf
import transformers
import numpy as np
import pandas as pd

# for plotting
import matplotlib.pyplot as plt

# if you want, not required
import seaborn as sns

Ensure that your versions match the ones below. If you are running this locally, install the packages with :

```bash
pip install transformers==4.35.0 tensorflow==2.14.0
```

If you get an error on import like:

```
NotFoundError: dlopen(ENVIRONMENT PATH/lib/python3.10/site-packages/tensorflow-plugins/libmetal_plugin.dylib, 0x0006): symbol not found in flat namespace '__ZN10tensorflow8internal10LogMessage16VmoduleActivatedEPKci'
```

This means that you are on a mac and you don't have the right `tensorflow-metal` [version installed](https://pypi.org/project/tensorflow-metal/):


```bash
pip install tensorflow-metal==1.1.0
```

Check your versions with the following code:


In [ ]:
# Version Info
print("Tensforflow Version : " ,tf.__version__)
print("Transformers Version : " ,transformers.__version__)

## TASK 2: The Data

We will be using reviews from the UCI Machine Learning Repository

URL : https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences

This should have three files :

1. imbd_labelled.txt
2. amazon_cells_labelled.txt
3. yelp_labelled.txt

We use the tensorflow utils to download and extract the data.

In [ ]:
# make a data folder
!mkdir -p data

# download the data
out_path = tf.keras.utils.get_file(origin="https://archive.ics.uci.edu/static/public/331/sentiment+labelled+sentences.zip",extract=True,cache_dir="data")
print("\n",out_path)

Let's take a peek at the data with `pandas`

In [ ]:
# viewing the data
imdb_data = "data/datasets/sentiment labelled sentences/imdb_labelled.txt"
imdb_df = pd.read_csv(imdb_data, sep="\t", header=None)
imdb_df

Models usually take batches of the same size. It would be useful to check the distribution of the lengths of the sentences in the dataset. (We will need to define a max length of sentences to use.)

In [ ]:
# TODO: plot the length of the reviews
# hint: use the .str.len() method on the first column of the dataframe to get the length of the sentences out
# y axis should be the count/number of reviews
# x axis should be the length of the reviews
# a histogram is a good choice here for visualization
# label your axes!

def display_histogram(data, title="IMDb Reviews"):
    lengths = [len(review) for review in data.loc[:,0]]

    x = [length for length in lengths]

    plt.hist(x,bins=30)
    plt.ylabel("Count")
    plt.xlabel("Review Length")
    plt.title(title)
    plt.show()

display_histogram(imdb_df)

**TODO:** Explore the other two data files, similarly to what we have done above (and what we've done on our homeworks).

In [ ]:
# Other data files explored here

## TASK 3: Setting BERT up

__IMPORTANT__:
BERT uses two special tokens while tokenization, they are [CLS] and [SEP]. The [CLS] token is used to capture the overall representation of the input sequence for classification tasks, while the [SEP] token is used to indicate sentence boundaries and separate different segments of text, enabling BERT to understand relationships between sentences. These tokens play a vital role in enabling BERT to handle a wide range of natural language processing tasks effectively.

Let us now begin by understanding the Tokenizer used by BERT and the BERT Model Inputs and Outputs

In [ ]:
# import specific models
from transformers import BertTokenizer, TFBertModel

# define a max length constant
MAX_LENGTH = 256

In [ ]:
# downloads a bunch of stuff. On our computer, this took ~half a minute
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = TFBertModel.from_pretrained('bert-base-uncased')

# expect to see:
# Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
# - This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
# - This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
# All the weights of TFBertModel were initialized from the PyTorch model.
# If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.

In [ ]:
# TODO: print out a summary of your downloaded bert model


Notice how there are 109M parameters. These parameters have already been trained. We want to make sure these are not trained when we add on our own components later. (We'll "freeze" them.)

1. What kind of text data do these weights encode? (Scour the internet and describe the contents of the data used to train BERT)
__YOUR ANSWER HERE__

## The Tokenizer

In [ ]:
# TODO: print out the first 10 tokens in the tokenizer's vocabulary, the
# last ten tokens, and ten tokens from somewhere in the middle of the vocabulary
# print out the things necessary to answer the questions below

# tokenizer.vocab will access the vocabulary
# tokenizer.vocab.keys() will give you the words in the vocabulary. You can convert this to a list to index into it.
# tokenizer.vocab[string] will give you the token id for the string
# tokenizer.decode(id) will give you the string for the token id

print("Vocab length is : ",len(tokenizer.vocab))

2. What is the token associated with id 2897? __YOUR ANSWER HERE__
3. What is the token associated with id 102? __YOUR ANSWER HERE__
4. What is the token associated with id 103? __YOUR ANSWER HERE__
5. Is your name in BERT's vocabulary (make sure to used the lowercase version, e.g. "mai")? If yes, what is it's id?  __YOUR ANSWER HERE__

In [ ]:
# example of tokenizing a sentence with the bert tokenizer
input_text = "Hello, my dog is cute, no really"
out = tokenizer(input_text,return_tensors="tf")
for key in out:
    print(key,": ",out[key])

# notice the differences between the input_text and what is printed out here
print(tokenizer.decode(out["input_ids"][0]))

Notice the different keys returned by the tokenizer. The model can take as input all three keys or just one of them the "input_ids".
Since the model requires a fixed length input, we will need to pad the sequences to the same length. Let us pad and turn the sequences into length 15 as an example.

In [ ]:
# TODO: set the max_length and truncation parameters of the tokenizer
# the behavior that you want to end up with is a tokenizer that produces tensors that
# are never over 15. You are not providing paired input

# Tokenizer documentation: https://huggingface.co/docs/transformers/main_classes/tokenizer

input_text = "Hello, my dog is not cute, no really"

# TODO: update this line
out = tokenizer(input_text,return_tensors="tf")


for key in out:
    print(key,": ",out[key])
print(tokenizer.decode(out["input_ids"][0]))

The tokenzier also returns multiple outputs for multiple sentences. This is useful for batching.

NOTE : The max length is 512 for BERT. We will use 128 or 256 for this lab (our `MAX_LENGTH` parameter)

In [ ]:
# TODO: make a list of sentences that you want to tokenize with at least 3 sentences that are padded to MAX_LENGTH
# make sure that you can tokenize them all at once by looking at the shapes of the tensors that have been output


# here's some code to look at the shapes of the tensors that have been output
for key in batch_x:
    print(key,": ",batch_x[key].shape)

In [ ]:
# run bert on the input_ids
out = bert(batch_x["input_ids"])

# display what the keys are for us
out.keys()

In [ ]:
# take a look at the shape of the last_hidden_state
out['last_hidden_state'].shape

The model returns two tensors of interest. One is the pooled output and the other is contextual hidden state over every token. We will use the hidden state associated with the first token [CLS] for classification. Additional strategies involve averaging the non padding states. The pooler output is a summary of the hidden states.

5. Investigate the shape of `last_hidden_state`. What is the __meaning__ of each dimension? (why are each dimension numbered as they are/what do they correspond to? You may need to go investigate some documentation to fully answer this!) __YOUR ANSWER HERE__

In [ ]:
# we can also run bert with unpacked inputs
out = bert(**batch_x)

# or with them as "input_ids" explicitly
out = bert(input_ids=batch_x["input_ids"])

print("First token [CLS] :")
out["last_hidden_state"][:,0,:]

## Create a Data Generator

In [ ]:
# define a batch size for our experiments
BATCH_SIZE = 4
# define a percentage of the data to use for training
SPLIT_PC = .80

# TODO: caluculate the last index for the training data
END =

In [ ]:
#  TODO: get the training data and testing data set up


# TODO: add print statements to verify the sizes of your training/testing data are correct



In [ ]:
# data generator for the model
def data_generator(sentences: np.array,labels: np.array,batch_size: int) -> (dict,tf.Tensor):
    i = 0
    while True:
        batch_x = []
        batch_y = []
        # TODO: append batch_size number of sentences and labels to batch_x and batch_y
        # Make sure that you don't re-use sentences and labels that you've already put into batches!
        YOU PROBABLY WANT A LOOP HERE

        # TODO: tokenize the batch_x, padding to MAX_LENGTH, and truncating to MAX_LENGTH
        batch_x = YOUR CODE HERE

        # debugging prints (make sure that these are commented out when you actually train your model)
        # should be (batch_size, MAX_LENGTH)
        # print(batch_x['input_ids'].shape)

        # convert our ys into the appropriate tensor
        batch_y = tf.convert_to_tensor(batch_y)

        # debugging prints (make sure that these are commented out when you actually train your model)
        # should be (batch_size,)
        # print(batch_y.shape)
        yield dict(batch_x), batch_y

train_data = data_generator(train_sentences,train_labels,BATCH_SIZE)
test_data = data_generator(test_sentences,test_labels,BATCH_SIZE)

**"Bonus" Question** - How would you implement randomness in the generator above. (HINT `numpy.random.choice` and its `replace` option)

(This is bonus in the sense that it is a good exercise to think about, not in the sense of us giving you extra credit. Feel free to talk to Prof. Mai more about the intersection of equity and extra credit.)

In [ ]:
# TODO: Take a look at the contents of tmp_batch_x and tmp_batch_y and report the shapes of the `input_ids`
# and the y label tensor.
# make sure that the shapes are what you expect them to be
# (take a look at the comments in the data_generator code)

tmp_batch_x,tmp_batch_y = next(train_data)



## TASK 4: Build the Model

Remember how you used concatenated embeddings for your NN model for HW5. The model we design in this lab will take as input token_ids (batch_size, embedding_dim) , run it through a non-trainable bert, extract the 768 dim vector associated with the [CLS] token of each sentence in a batch, this 768 dim vector will play the role of our concatenated embeddings. The main take away is this : Any input size up to 512 will return a 768 dim vector we can use as an embedding for the entire sentence.

In [ ]:
# Build the model
# This takes < 15 sec to run on our computer

bert_model = TFBertModel.from_pretrained('bert-base-uncased',output_attentions = False,return_dict=False)
# we do not need attention outputs
# we want to return tuples since they are easier to access

bert_model.trainable = False
# setting trainable to false ensures
# we do not update its weights
model_ = tf.keras.Sequential([
    bert_model,
    tf.keras.layers.Lambda(lambda x: x[0][:,0,:]), # https://keras.io/api/layers/core_layers/lambda/
    tf.keras.layers.Dense(50,activation="relu"),
    tf.keras.layers.Dense(1,activation="sigmoid")
])

model_.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

1. What does the `Lambda` layer do? Why do we need it? (Read the documentation and investigate. Think carefully about what `x[0][:,0,:]` means. `x[0]` here is a numpy array.) __YOUR ANSWER HERE__
2. What weights will you be training in this model? __YOUR ANSWER HERE__
2. What weights will you __not__ be training in this model? __YOUR ANSWER HERE__

## Train the model

We use `.fit` here

We will also be adding a validation data generator and validation steps.
This will allow us to check accuracy on the test_data wile we train over each epoch.

For this part, you'll be training in some different configurations and __recording your results__. (don't forget to write these down!)

In [ ]:
# 2 epochs takes ~2 and a half minutes on our computer
model_.fit(
    train_data,
    epochs=2,
    batch_size=BATCH_SIZE,
    steps_per_epoch=len(train_sentences)//BATCH_SIZE,
    validation_data=test_data,
    validation_steps=BATCH_SIZE*4,
    validation_batch_size=BATCH_SIZE
)

TODO: Now train the model on the other two files present in the data folder and report your results (make sure that you train your model from scratch). You will want to experiment with different `MAX_LENGTH`s, `batch_size`s, the number of dense layers you have, the number of hidden units per layer.

In general, the more layers corresponds to the more levels of abstract information that your model will be able to extract/represent. The more hidden units (the "wider") your network has, the more information it will be able to memorize.

4. Using the "base" model settings (the default values we've set for you), record the `val_accuracy` after 2 epochs for each training data set:
    1. imdb: __YOUR ANSWER HERE__
    2. amazon: __YOUR ANSWER HERE__
    3. yelp: __YOUR ANSWER HERE__

5. Choose one dataset. For at least 4 different combinations of settings, record the following five pieces of information:
    1. training data (imdb, amazon, yelp) (this will be the same for all 4 combinations)
    2. epochs
    3. dimensions of your network (# of layers, # of hidden units per layer) — this is the part after the bert model
    4. `MAX_LENGTH` value
    5. `batch_size` value
    6. time to train
    7. `val_accuracy` in the end

__YOUR RESULTS HERE (format these nicely!)__


6. What did your experiments show you? __YOUR ANSWER HERE__